# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
pip install citipy

Note: you may need to restart the kernel to use updated packages.


In [19]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [20]:

# Store Part I results into DataFrame
cities_df = pd.read_csv('cities.csv')

cities_df

,City,Latitude,Longitude,Temperature,Maximum Temperature,Humidity,Cloudiness,Windspeed
0,katsuura,35.13,140.30,57.27,59.00,54.0,75.0,23.04
1,new norfolk,-42.78,147.06,56.05,57.99,93.0,90.0,3.36
2,rikitea,-23.12,-134.97,78.66,78.66,73.0,27.0,4.68
3,sitka,57.05,-135.33,33.26,33.80,50.0,1.0,5.82
4,georgetown,5.41,100.34,81.34,82.40,83.0,40.0,3.36
...,...,...,...,...,...,...,...,...
553,jequitinhonha,-16.43,-41.00,88.48,88.48,47.0,40.0,4.85
554,erenhot,43.65,111.98,33.76,33.76,42.0,0.0,11.03
555,pontiac,42.64,-83.29,47.41,50.00,65.0,90.0,10.29
556,pauini,-7.71,-66.98,87.39,87.39,67.0,89.0,3.33


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [21]:
# Humidity Heatmap

# Use the Lat and Lng as locations and Humidity as the weight.
locations = cities_df[["Latitude", "Longitude"]].astype(float)

humidity = cities_df["Humidity"].astype(float)

# Configure gmaps
gmaps.configure(api_key=g_key)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity)

# Add Heatmap layer to map
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [22]:
# Create new DataFrame fitting weather criteria
# A max temperature lower than 80 degrees but higher than 60.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

cities_filter_df = cities_df.loc[((cities_df['Temperature'] >= 60) & (cities_df['Temperature'] < 80)) & 
                                 (cities_df['Windspeed'] <= 10) & 
                                 (cities_df['Cloudiness'] == 0)]

# Add a "Hotel Name" column to the DataFrame.
cities_filter_df['Country'] = ""
cities_filter_df['Hotel Name'] = ""

cities_filter_df.head()


C:\Users\12034\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\Users\12034\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,City,Latitude,Longitude,Temperature,Maximum Temperature,Humidity,Cloudiness,Windspeed,Country,Hotel Name
22,matara,5.95,80.54,76.80,76.80,76.0,0.0,7.83,,
28,chipinge,-20.19,32.62,65.19,65.19,50.0,0.0,4.54,,
36,abu dhabi,24.47,54.37,74.35,75.99,78.0,0.0,4.70,,
77,ruwi,23.60,58.55,75.20,75.20,73.0,0.0,2.24,,
96,selma,36.57,-119.61,63.66,66.99,68.0,0.0,4.70,,


In [24]:
# Hotel Map

# Set parameters to search for hotels with 5000 meters.
target_search = "hotel"
target_radius = 5000
target_type = "lodging"

for index, row in cities_filter_df.iterrows():
    # base url for google places call
    base_url_place = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # base url for reverse geocode call
    base_url_geocode = "https://maps.googleapis.com/maps/api/geocode/json"
    
        # set up a parameters dictionary for the reverse geocode call
    try:
        params_geocode = {
            "address": row['City'],
            "key": g_key
        }
    
        # Hit the Google Places API for each city's coordinates.
        response_geocode = requests.get(base_url_geocode, params=params_geocode)
    except Exception as e:
        print(e)
    
    # convert response to json
    try:
        geocode_data = response_geocode.json()
    except Exception as e:
        print(e)
    
    # Select the block with "country" in "types"
    try:
        country_range = np.arange(len(geocode_data['results'][0]['address_components']))
        for number in country_range:
            if 'country' in geocode_data['results'][0]['address_components'][number]['types']:
                country = geocode_data['results'][0]["address_components"][number]['short_name']
                cities_filter_df.loc[index, "Country"] = country
    except Exception as e:
        cities_filter_df.loc[index, "Country"] = np.NaN
        print(e)

    # set up a parameters dictionary for google places
    try:
        lat = geocode_data["results"][0]["geometry"]["location"]["lat"]
        cities_filter_df.loc[index, "Latitude"] = lat
    except Exception as e:
        cities_filter_df.loc[index, "Latitude"] = np.NaN
        print(e)
    try:
        lng = geocode_data["results"][0]["geometry"]["location"]["lng"]
        cities_filter_df.loc[index, "Longitude"] = lng
    except Exception as e:
        cities_filter_df.loc[index, "Longitude"] = np.NaN
        print(e)
        
    params_place = {
        "location": f"{lat}, {lng}",
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }
    
    # run a request using our params dictionary
    try:
        response_place = requests.get(base_url_place, params=params_place)
    except Exception as e:
        print(e)
        
    # convert response to json
    try:
        places_data = response_place.json()
    except Exception as w:
        print(e)
        
    # select name of first hotel
    try:
        hotel = places_data['results'][0]["name"]
    # Store the first Hotel result into the DataFrame.
        cities_filter_df.loc[index, "Hotel Name"] = hotel
    except Exception as e:
        cities_filter_df.loc[index, "Hotel Name"] = np.NaN
        print(e)

    time.sleep(1)

cities_filter_df.head(5)

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out o

,City,Latitude,Longitude,Temperature,Maximum Temperature,Humidity,Cloudiness,Windspeed,Country,Hotel Name
22,matara,NaN,NaN,76.80,76.80,76.0,0.0,7.83,NaN,NaN
28,chipinge,NaN,NaN,65.19,65.19,50.0,0.0,4.54,NaN,NaN
36,abu dhabi,NaN,NaN,74.35,75.99,78.0,0.0,4.70,NaN,NaN
77,ruwi,NaN,NaN,75.20,75.20,73.0,0.0,2.24,NaN,NaN
96,selma,NaN,NaN,63.66,66.99,68.0,0.0,4.70,NaN,NaN


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [25]:

# Store into variable named hotel_df
hotel_df = cities_filter_df[['City', 'Country', 'Hotel Name']]
hotel_df = pd.merge(cities_df, hotel_df, on='City', suffixes=('', ''))
hotel_df

,City,Latitude,Longitude,Temperature,Maximum Temperature,Humidity,Cloudiness,Windspeed,Country,Hotel Name
0,matara,5.95,80.54,76.80,76.80,76.0,0.0,7.83,NaN,NaN
1,chipinge,-20.19,32.62,65.19,65.19,50.0,0.0,4.54,NaN,NaN
2,abu dhabi,24.47,54.37,74.35,75.99,78.0,0.0,4.70,NaN,NaN
3,ruwi,23.60,58.55,75.20,75.20,73.0,0.0,2.24,NaN,NaN
4,selma,36.57,-119.61,63.66,66.99,68.0,0.0,4.70,NaN,NaN
5,blackwater,-23.58,148.88,68.29,68.29,82.0,0.0,5.64,NaN,NaN
6,hambantota,6.12,81.12,73.40,73.40,100.0,0.0,2.24,NaN,NaN
7,luderitz,-26.65,15.16,62.60,62.60,72.0,0.0,6.93,NaN,NaN
8,matay,28.42,30.78,72.05,72.05,27.0,0.0,6.87,NaN,NaN
9,san quintin,30.48,-115.95,65.16,65.16,62.0,0.0,7.85,NaN,NaN


In [27]:
# Plot markers on top of the heatmap.

# Humidity Heatmap

# Use the Lat and Lng as locations and Humidity as the weight.
locations = hotel_df[["Latitude", "Longitude"]].astype(float)
humidity = hotel_df["Humidity"].astype(float)
marker_locations = hotel_df[["Latitude", "Longitude"]].loc[hotel_df['Hotel Name'].notna()]

name = hotel_df['Hotel Name'].loc[hotel_df['Hotel Name'].notna()].tolist()
city = hotel_df['City'].loc[hotel_df['Hotel Name'].notna()].tolist()
country = hotel_df['Country'].loc[hotel_df['Hotel Name'].notna()].tolist()


# Configure gmaps
fig = gmaps.figure()

# Add Heatmap layer to map

heat_layer = gmaps.heatmap_layer(locations, 
                                 weights=humidity)

In [28]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [29]:
# Add marker layer ontop of heat map
name_markers = gmaps.marker_layer(locations, 
                                  info_box_content = hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(name_markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))